In [92]:
import pandas as pd
from sqlalchemy import create_engine
import os
import numpy as np

os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Project-2/Sport-Salaries/')


In [93]:
### Extract CSVs into DataFrames
MLS_file = "MLS_Salaries.csv"
MLS_df = pd.read_csv(MLS_file)
MLS_df.head()


,Season,Club,First Name,Last Name,Position,Total Compensation,Base Salary,name
0,2018,TOR,Sebastian,Giovinco,F,7115556,5600000,Sebastian Giovinco
1,2018,TOR,Michael,Bradley,M,6500000,6000000,Michael Bradley
2,2018,LAFC,Carlos,Vela,F,6292500,4500000,Carlos Vela
3,2018,CHI,Bastian,Schweinsteiger,M,6100000,6100000,Bastian Schweinsteiger
4,2018,LA,Giovani,dos Santos,F,6000000,4250000,Giovani dos Santos


In [95]:
# Create a filtered dataframe from specific columns
mls_cols = ["name", "Position", "Club", "Base Salary", "Season"]
mls_transformed = MLS_df[mls_cols].copy()

# Rename the column headers
mls_transformed = mls_transformed.rename(columns={"name": "player_name",
                                                          "Position": "position",
                                                          "Club": "team",
                                                          "Base Salary": "avg_salary",
                                                          "Season": "year"})

# Clean the data by dropping NA's, clean avg_salary, and setting the index
mls_transformed = mls_transformed.dropna(axis = 0, how ='any') 
mls_transformed = mls_transformed.sort_values('team', ascending=False).drop_duplicates(subset=['player_name', 'position', 'year'])

#mlb_transformed_cond = mlb_transformed['avg_salary'] != "$NaN"
#mlb_transformed = mlb_transformed[mlb_transformed_cond]

mls_transformed.set_index("year", inplace=True)
mls_transformed.sort_values(by=['avg_salary'])
mls_transformed.head()


,player_name,position,team,avg_salary
year,,,,
2013,Joe Cannon,GK,VAN,180500
2014,Johnny Leveron,D,VAN,80000
2018,Doneil Henry,D,VAN,140004
2014,Russell Teibert,M,VAN,70000
2018,Erik Hurtado,F/M,VAN,150000


In [96]:
#connect to postgres
connection_string = "postgres:akopova@localhost:5433/sports_db"
engine = create_engine(f'postgresql://{connection_string}')


In [97]:
#create NFL player table. Says there is an error, but table has been created.
pd.read_sql_query('CREATE TABLE mls_player_salary(year INT, player_name VARCHAR(500), position VARCHAR(100), team VARCHAR(100), avg_salary INT, PRIMARY KEY (year, player_name, position))', con=engine).head()



ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [98]:
# Confirm table
engine.table_names()


['nfl_player_salary',
 'mlb_player_salary',
 'nba_player_salary',
 'mls_player_salary']

In [99]:
#Load into NFL player Salary
mls_transformed.to_sql(name='mls_player_salary', con=engine, if_exists='append', index=True)


In [100]:
pd.read_sql_query('select * from mls_player_salary', con=engine).head()


,year,player_name,position,team,avg_salary
0,2013,Joe Cannon,GK,VAN,180500
1,2014,Johnny Leveron,D,VAN,80000
2,2018,Doneil Henry,D,VAN,140004
3,2014,Russell Teibert,M,VAN,70000
4,2018,Erik Hurtado,F/M,VAN,150000
